# Notebook de développement du modèle

Ce dataset comporte peu de données pour chaque classe (entre 60 et 100). <br>
Ainsi il faudra adapter les méthodes d'apprentissages.

## Data Management

In [19]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import glob

In [ ]:
liste_poissons=['apognon','epinephelus_marginatus','girelle_paon','pempheris_vanicolensis','sparisoma_cretense']

### Renommer les fichiers

In [20]:
'''
for espece in liste_poisson:
    root = r"data/{}".format(espece)
    print(root)
    count = 0
    for dirs, subdir, files in os.walk(root):
        for file in files:
            os.rename(root+"/"+file,root+"/{}{}.jpg".format(espece,count))
            count+=1
'''

'\nroot = r"data/girelle_paon"\nprint(root)\ncount = 0\nfor dirs, subdir, files in os.walk(root):\n    for file in files:\n        os.rename(root+"/"+file,root+"/girelle_paon{}.jpg".format(count))\n        count+=1\n'

### Redimmensionner les images

In [21]:
'''
for espece in liste_poisson:
    files = glob.glob('./data/{}/*'.format(espece))

    for file in files:
        img = Image.open(file)
        img_resize = img.resize((150, 100))
        root, ext = os.path.splitext(file)
        basename = os.path.basename(root)
        print(basename)
        img_resize.save(os.path.join('./data/{}/'.format(espece), basename+'.jpg'))
'''

"\nfiles = glob.glob('./data/sparisoma_cretense/*')\n\nfor file in files:\n    img = Image.open(file)\n    img_resize = img.resize((150, 100))\n    root, ext = os.path.splitext(file)\n    basename = os.path.basename(root)\n    print(basename)\n    img_resize.save(os.path.join('./data/sparisoma_cretense/', basename+'.jpg'))\n"

### Verifier que les dimensions de chaque images soient bien les mêmes

In [33]:
'''
for espece in liste_poissons:
    files = glob.glob('./data/{}/*'.format(espece))
    compteur=0
    for f in files:
        im = np.array(Image.open(f))

        if im.dtype!='uint8':
            print("mauvais type"+str(compteur))
            print(im.dtype)
        if im.shape!=(100,150,3):
            print("l'image {} n'a pas les bonnes dimensions!".format(compteur))
            print(im.shape)
        compteur+=1
'''

### Conversion des images en niveaux de gris

In [38]:
'''
for espece in liste_poissons:
    files = glob.glob('./data/grey/{}/*'.format(espece))
    for file in files:
        img = Image.open(file).convert('L')
        root, ext = os.path.splitext(file)
        basename = os.path.basename(root)
        img.save(os.path.join('./data/grey/{}/'.format(espece), basename+'.jpg'))
'''

### Normalisation